In [1]:
#import Important liabraries
import pandas as pd
import numpy as np


In [2]:
#Read Downloaded dataset
df = pd.read_excel('Dry_Bean_Dataset.xlsx')

In [3]:
df.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


In [4]:
#findin the shape of dataset
df.shape

(13611, 17)

In [5]:
#use sklearn model to split dataset into training and test
from sklearn.model_selection import train_test_split

In [6]:
train_data, test_data = train_test_split(df, test_size=0.20, random_state=42)

In [7]:
len(train_data)

10888

In [8]:
len(test_data)

2723

In [9]:
#splotting train_data into train and validation datasets seperatly

train2, val = train_test_split(train_data, test_size=0.20, random_state=42)

In [10]:
#our training data length
len(train2)

8710

In [11]:
#Our validation data length
len(val)

2178

In [12]:
#our testing data length
len(test_data)

2723

In [13]:
type(val)

pandas.core.frame.DataFrame

In [14]:
#'dict', 'list', 'series', 'split', 'records', 'index' 
#convet all datasets in to dictionary for insertion into mongodb.

train_data = train2.to_dict(orient='list')
test_data = test_data.to_dict(orient='list')
val_data=val.to_dict(orient='list')

In [1]:
#import datasets into Mongo-db
import pymongo

In [2]:
myclient = pymongo.MongoClient('mongodb://localhost:27017/')

In [3]:
#create database in mongo db
mydb = myclient['project080422']

In [4]:
#create collections in mydb databases
mycol_train = mydb['train_data']
mycol_test = mydb['test_data']
mycol_val = mydb['validation_data']

In [19]:
#Insert data into collection
train = mycol_train.insert_one(train_data)
test = mycol_test.insert_one(test_data)
val = mycol_val.insert_one(val_data)
print("all data is inserted into Mongodb database>>>>>>>>>>>>..")

all data is inserted into Mongodb database>>>>>>>>>>>>..


In [29]:
all_records = mycol_test.find_one()

In [27]:
print(all_records)

{'_id': ObjectId('624ff67f5f8f5b185472743e'), 'Area': [42339, 68247, 37856, 33143, 29925, 38489, 89592, 42379, 203536, 27306, 33380, 47884, 41451, 53684, 25148, 174621, 33409, 45954, 68309, 21961, 93503, 74783, 54010, 29551, 138777, 35781, 70938, 23090, 41980, 93494, 55487, 36272, 38825, 29821, 52398, 155439, 44111, 25306, 39765, 57356, 53347, 34207, 45428, 54187, 62465, 41739, 65974, 24887, 44522, 35677, 49946, 77418, 41565, 78152, 56318, 57778, 40854, 30512, 171914, 58386, 159048, 41443, 44171, 45454, 38104, 48436, 42008, 30774, 51088, 175965, 25726, 52153, 42122, 36109, 60516, 49198, 42853, 48915, 33494, 50908, 47114, 21314, 33171, 66985, 47371, 34827, 55607, 25423, 43021, 70445, 83112, 25232, 53473, 47360, 40514, 183093, 40127, 52856, 35664, 46436, 39175, 42312, 47969, 43968, 68879, 41435, 56069, 64295, 42438, 30017, 37209, 31007, 43676, 40363, 51896, 43976, 47122, 44474, 64530, 80953, 24590, 36076, 29687, 78848, 46063, 40896, 66931, 40752, 41040, 43329, 47769, 34445, 29702, 30908,

In [25]:
for i in all_records:
    print(i)

_id
Area
Perimeter
MajorAxisLength
MinorAxisLength
AspectRation
Eccentricity
ConvexArea
EquivDiameter
Extent
Solidity
roundness
Compactness
ShapeFactor1
ShapeFactor2
ShapeFactor3
ShapeFactor4
Class


In [9]:
import pandas as pd

In [14]:
df = pd.DataFrame(dict_rec)

In [15]:
df.head()

,_id,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,624ff67f5f8f5b185472743e,"[42339, 68247, 37856, 33143, 29925, 38489, 895...","[741.2259999999999, 1088.754, 708.715999999999...","[260.19933054636994, 370.36814568575124, 248.4...","[207.30639412352278, 237.86379197694248, 194.3...","[1.2551437771443321, 1.5570597887452042, 1.278...","[0.6043466663528361, 0.7665068799192755, 0.622...","[42676, 70172, 38232, 33377, 30321, 38782, 906...","[232.18029435019253, 294.7792041673575, 219.54...","[0.7712021857923498, 0.7676827896512936, 0.744...","[0.9921032899053333, 0.9725674058028844, 0.990...","[0.968387491136667, 0.7234922815413962, 0.9471...","[0.8923170319564517, 0.7959086319952331, 0.883...","[0.006145618237236825, 0.005426878041316852, 0...","[0.0024033779461940423, 0.0013433299075862079,...","[0.7962296855195715, 0.6334705504845237, 0.780...","[0.9993827349574785, 0.9863515215137404, 0.998...","[SEKER, BARBUNYA, SEKER, SEKER, DERMASON, SEKE..."
